In [1]:
import os
annotated = sorted(os.listdir('data_test/images/'))[:-1]
unannotated = sorted(os.listdir('data_test/images/unannotated/'))

In [2]:
import pandas as pd
df = pd.DataFrame()
for i in sorted(os.listdir('data_test/annotation/dir')):
    tmp = pd.read_csv('data_test/annotation/dir/' + i, header=None)
    tmp.columns = ['filename', 'dir']
    tmp['filename'] = tmp['filename'].apply(
        lambda x: 'data_test/images/' + i.split('_')[0] + '_' + str(x) + '.jpg' if int(i.split('_')[0]) <= 35 
        else 'data_test/images/unannotated/' + i.split('_')[0] + '_' + str(x) + '.jpg')
    df = pd.concat([df, tmp])

In [3]:
y = df

In [4]:
import cv2

X = [(i, cv2.resize(cv2.imread(i), (224, 224))) for i in y.filename]

In [5]:
y = y.sort_values(by='filename').dir

In [6]:
X = list(map(lambda x: x[1], sorted(X, key=lambda x: x[0])))

In [7]:
from keras.applications.mobilenet import MobileNet
from keras.optimizers import Adam
from keras.layers import Dense, AveragePooling2D, Flatten
from keras.models import Model

model = MobileNet(input_shape=(224, 224, 3), classes=6, include_top=False)
pool = AveragePooling2D()(model.output)
flat = Flatten()(pool)
fc1 = Dense(32, activation='relu')(flat)
out = Dense(6)(fc1)

model = Model(model.input, out)

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.


In [8]:
model.compile(Adam(lr=3e-4), loss='categorical_crossentropy')

In [9]:
import numpy as np
X = np.stack(X)

In [10]:
from keras.preprocessing.image import ImageDataGenerator

X = X / 255
X *= 2
X -= 1

In [11]:
label_map = dict(list(zip(set(y), range(6))))
y = y.apply(lambda x: label_map[x])

In [12]:
from keras.utils import to_categorical

y = to_categorical(y)


In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [15]:
datagen = ImageDataGenerator(brightness_range=(0.3, 0.7),
                             zoom_range=0.2,
                             shear_range=0.2)

In [16]:
from keras.callbacks import TensorBoard, EarlyStopping, ReduceLROnPlateau

tb = TensorBoard()
es = EarlyStopping(patience=3)
sh = ReduceLROnPlateau()

call = [tb, es, sh]

In [17]:
model.fit_generator(datagen.flow(X, y, batch_size=32),
                    steps_per_epoch=len(X) / 32, epochs=100,
                   callbacks=call)

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
  4/109 [>.............................] - ETA: 13:49 - loss: 2.4460

KeyboardInterrupt: 

In [ ]:
print(model.evaluate(X_test, y_test))

model.save